In [4]:
import pandas as pd
import numpy as np
import os

dir_list = [
    "/media/bruno/f71fc4fd-abcd-4a4b-9fb1-6a8899041c1c/FX-Training-Data/Sorted/EURUSD",
    "/media/bruno/f71fc4fd-abcd-4a4b-9fb1-6a8899041c1c/FX-Training-Data/Sorted/GBPUSD",
    "/media/bruno/f71fc4fd-abcd-4a4b-9fb1-6a8899041c1c/FX-Training-Data/Sorted/GBPJPY",
]

# Iterate over the directories
for directory in dir_list:
    # Get the names of all the CSV files in the directory
    files = [f for f in os.listdir(directory) if f.endswith('_modified.csv')]

    # Iterate over the CSV files
    for file in files:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(os.path.join(directory, file))

        # Calculate the wick lengths
        df['upper_wick_length'] = np.where(df['p_open'] == df['p_close'], 0, (df['p_high'] - df[['p_open', 'p_close']].max(axis=1)) / (df['p_high'] - df['p_low']))
        df['lower_wick_length'] = np.where(df['p_open'] == df['p_close'], 0, (df[['p_open', 'p_close']].min(axis=1) - df['p_low']) / (df['p_high'] - df['p_low']))

        # Scale the wick lengths to a range of 0 to 5
        df['upper_wick_length'] = np.where(df['upper_wick_length'] >= 0.95, 0,
                                        np.where(
                                        (df['upper_wick_length'] >= 0.8) & (df['upper_wick_length'] < 0.95), 1,
                                        np.where(
                                        (df['upper_wick_length'] >= 0.6) & (df['upper_wick_length'] < 0.8), 2,
                                        np.where(
                                        (df['upper_wick_length'] >= 0.4) & (df['upper_wick_length'] < 0.6), 3,
                                        np.where(
                                        (df['upper_wick_length'] >= 0.2) & (df['upper_wick_length'] < 0.4), 4,
                                        np.where(
                                        (df['upper_wick_length'] >= 0) & (df['upper_wick_length'] < 0.2), 5, np.nan))))))

        df['lower_wick_length'] = np.where(df['lower_wick_length'] >= 0.95, 0,
                                        np.where(
                                        (df['lower_wick_length'] >= 0.8) & (df['lower_wick_length'] < 0.95), 1,
                                        np.where(
                                        (df['lower_wick_length'] >= 0.6) & (df['lower_wick_length'] < 0.8), 2,
                                        np.where(
                                        (df['lower_wick_length'] >= 0.4) & (df['lower_wick_length'] < 0.6), 3,
                                        np.where(
                                        (df['lower_wick_length'] >= 0.2) & (df['lower_wick_length'] < 0.4), 4,
                                        np.where(
                                        (df['lower_wick_length'] >= 0) & (df['lower_wick_length'] < 0.2), 5, np.nan))))))

        # Write the modified DataFrame to a new CSV file
        output_file = os.path.join(directory, file)
        df.to_csv(output_file, index=False)